In [1]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math

In [2]:
def get_intensity_pictures(img,composite,outer_bool=False):  
    _ , thresh = cv2.threshold(composite, 5, 10, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    black = np.full(thresh.shape, 0, dtype=np.uint8)
    if outer_bool:
        return (black , composite, composite)

    white_outer_blackbg = cv2.drawContours(black, contours, -1, 255, -1)
    clean_composite = cv2.bitwise_and(composite, white_outer_blackbg)

    left = (math.inf,0)
    right = (0,0)
    top = (0, math.inf)
    bottom = (0,0)
    threshold_intensity = 10
    #Find left most edge
    for y, row in enumerate(clean_composite):
        for x, pixel in enumerate(row):
            if pixel >= threshold_intensity and x < left[0]:
                left = (x,y)
            #Find right most edge
            if pixel >= threshold_intensity and x > right[0]:
                right = (x,y)
            #Find top most edge
            if pixel >= threshold_intensity and y < top[1]:
                top = (x,y)
            #Find bottom most edge
            if pixel >= threshold_intensity and y > bottom[1]:
                bottom = (x,y)

    #Given four points find top left and bottom right of the rectangle
    def find_top_left_bottom_right(p1,p2,p3,p4):
        top_left = (min(p1[0],p2[0],p3[0],p4[0]),min(p1[1],p2[1],p3[1],p4[1]))
        bottom_right = (max(p1[0],p2[0],p3[0],p4[0]),max(p1[1],p2[1],p3[1],p4[1]))
        return top_left, bottom_right

    def x_radius(p1,p2):
        return (p2[0]-p1[0])/2
    def y_radius(p1,p2):
        return (p2[1]-p1[1])/2

    top_left, bottom_right = find_top_left_bottom_right(left,right,top,bottom)

    left_2 = (left[0] + x_radius(top_left,bottom_right)/2, left[1])
    right_2 = (right[0] - x_radius(top_left,bottom_right)/2, right[1])
    top_2 = (top[0], top[1] + y_radius(top_left,bottom_right)/2)
    bottom_2 = (bottom[0], bottom[1] - y_radius(top_left,bottom_right)/2)
    top_left_2, bottom_right_2 = find_top_left_bottom_right(left_2,right_2,top_2,bottom_2) 
    center = (int((top_left_2[0] + bottom_right_2[0])/2), int((top_left_2[1] + bottom_right_2[1])/2))

    #Make a mask of the ellipse that sets everything outside the ellipse to 0
    mask = np.zeros(composite.shape, dtype=np.uint8)
    mask = cv2.ellipse(mask,
                         center,
                         (int(x_radius(top_left_2,bottom_right_2)), int(y_radius(top_left_2,bottom_right_2)))
                         , 0, 0, 360, (255,255 ,255), -1)

    #Apply the mask to the image
    masked_image = cv2.bitwise_and(composite, mask)

    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    # Draw contours + hull results
    mask_2 = np.zeros(thresh.shape, dtype=np.uint8)

    cv2.drawContours(mask_2, contours, -1, (255,255 ,255), -1)
    masked_image_2 = cv2.bitwise_and(composite, mask_2)

    #Remove inner from outer
    mask_1_1 = cv2.bitwise_not(mask)
    outer_only = cv2.bitwise_and(masked_image_2, mask_1_1)

    return (masked_image, outer_only, composite)

In [3]:
#Get matrix of pixel intensities for red and blue for inner and outer
def get_intensity_points(img,composite,outer_bool=False):  
    _,thresh = cv2.threshold(composite, 5, 10, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # composite = composite.astype(np.uint16)
    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    black = np.full(thresh.shape, 0, dtype=np.uint8)
    white_outer_blackbg = cv2.drawContours(black, contours, -1, 255, -1)

    clean_composite = cv2.bitwise_and(composite, white_outer_blackbg)

    left = (math.inf,0)
    right = (0,0)
    top = (0, math.inf)
    bottom = (0,0)
    threshold_intensity = 10
    #Find left most edge
    for y, row in enumerate(clean_composite):
        for x, pixel in enumerate(row):
            if pixel >= threshold_intensity and x < left[0]:
                left = (x,y)
            #Find right most edge
            if pixel >= threshold_intensity and x > right[0]:
                right = (x,y)
            #Find top most edge
            if pixel >= threshold_intensity and y < top[1]:
                top = (x,y)
            #Find bottom most edge
            if pixel >= threshold_intensity and y > bottom[1]:
                bottom = (x,y)

    #Given four points find top left and bottom right of the rectangle
    def find_top_left_bottom_right(p1,p2,p3,p4):
        top_left = (min(p1[0],p2[0],p3[0],p4[0]),min(p1[1],p2[1],p3[1],p4[1]))
        bottom_right = (max(p1[0],p2[0],p3[0],p4[0]),max(p1[1],p2[1],p3[1],p4[1]))
        return top_left, bottom_right

    def x_radius(p1,p2):
        return (p2[0]-p1[0])/2
    def y_radius(p1,p2):
        return (p2[1]-p1[1])/2

    top_left, bottom_right = find_top_left_bottom_right(left,right,top,bottom)

    color = (255, 0, 0)
    left_2 = (left[0] + x_radius(top_left,bottom_right)/2, left[1])
    right_2 = (right[0] - x_radius(top_left,bottom_right)/2, right[1])
    top_2 = (top[0], top[1] + y_radius(top_left,bottom_right)/2)
    bottom_2 = (bottom[0], bottom[1] - y_radius(top_left,bottom_right)/2)
    top_left_2, bottom_right_2 = find_top_left_bottom_right(left_2,right_2,top_2,bottom_2) 
    center = (int((top_left_2[0] + bottom_right_2[0])/2), int((top_left_2[1] + bottom_right_2[1])/2))

    ret, thresh = cv2.threshold(composite, 5, 10, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    black = np.full(thresh.shape, 0, dtype=np.uint8)
    white_outer_blackbg = cv2.drawContours(black, contours, -1, 255, -1)

    white = np.full(img.shape, 255, dtype=np.uint8)
    black_ellipse_whitebg= cv2.ellipse(white,
                         center,
                         (int(x_radius(top_left_2,bottom_right_2)), int(y_radius(top_left_2,bottom_right_2)))
                         , 0, 0, 360, 0, -1)

    outer_mask = cv2.bitwise_and(white_outer_blackbg, black_ellipse_whitebg)

    #Make a mask of the ellipse that sets everything outside the ellipse to 0
    inner_mask = np.full(img.shape, 0, dtype=np.uint8)
    inner_mask = cv2.ellipse(inner_mask,
                         center,
                         (int(x_radius(top_left_2,bottom_right_2)), int(y_radius(top_left_2,bottom_right_2)))
                         , 0, 0, 360, 255, -1)
    
    rows_ellipse, cols_ellipse  = np.where(inner_mask > 0)
    pts_ellipse = (rows_ellipse, cols_ellipse)

    rows_outer, cols_outer = np.where(outer_mask > 0)
    pts_outer = (rows_outer, cols_outer)

    #Get list of pixel intensities that are in the inner rectangle
    def get_pixel_intensities(img, pts):
        pixel_intensities = img[pts[0], pts[1]] 
        return pixel_intensities

    outer = get_pixel_intensities(img, pts_outer)
    inner = get_pixel_intensities(img, pts_ellipse)
    
    if outer_bool:
        pts_outer = np.where(white_outer_blackbg > 0)
        outer = get_pixel_intensities(img, pts_outer)
        inner = [0 for i in range(len(outer))]

    return (outer, inner)

In [4]:
def get_pictures_norm(img_red, img, composite, outer_bool = False):
    _ , thresh = cv2.threshold(composite, 5, 10, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #Find biggest contour
    contours = [max(contours, key = cv2.contourArea)]

    black = np.full(thresh.shape, 0, dtype=np.uint8)
    white_outer_blackbg = cv2.drawContours(black, contours, -1, 255, -1)
    clean_composite = cv2.bitwise_and(composite, white_outer_blackbg)

    left = (math.inf,0)
    right = (0,0)
    top = (0, math.inf)
    bottom = (0,0)
    threshold_intensity = 10

    #Find left most edge
    for y, row in enumerate(clean_composite):
        for x, pixel in enumerate(row):
            if pixel >= threshold_intensity and x < left[0]:
                left = (x,y)
            #Find right most edge
            if pixel >= threshold_intensity and x > right[0]:
                right = (x,y)
            #Find top most edge
            if pixel >= threshold_intensity and y < top[1]:
                top = (x,y)
            #Find bottom most edge
            if pixel >= threshold_intensity and y > bottom[1]:
                bottom = (x,y)

    #Given four points find top left and bottom right of the rectangle
    def find_top_left_bottom_right(p1,p2,p3,p4):
        top_left = (min(p1[0],p2[0],p3[0],p4[0]),min(p1[1],p2[1],p3[1],p4[1]))
        bottom_right = (max(p1[0],p2[0],p3[0],p4[0]),max(p1[1],p2[1],p3[1],p4[1]))
        return top_left, bottom_right

    def x_radius(p1,p2):
        return (p2[0]-p1[0])/2
    def y_radius(p1,p2):
        return (p2[1]-p1[1])/2

    top_left, bottom_right = find_top_left_bottom_right(left,right,top,bottom)

    left_2 = (left[0] + x_radius(top_left,bottom_right)/2, left[1])
    right_2 = (right[0] - x_radius(top_left,bottom_right)/2, right[1])
    top_2 = (top[0], top[1] + y_radius(top_left,bottom_right)/2)
    bottom_2 = (bottom[0], bottom[1] - y_radius(top_left,bottom_right)/2)
    top_left_2, bottom_right_2 = find_top_left_bottom_right(left_2,right_2,top_2,bottom_2) 
    center = (int((top_left_2[0] + bottom_right_2[0])/2), int((top_left_2[1] + bottom_right_2[1])/2))

    black = np.full(thresh.shape, 0, dtype=np.uint8)
    white_outer_blackbg = cv2.drawContours(black, contours, -1, 255, -1)

    white = np.full(img.shape, 255, dtype=np.uint8)
    black_ellipse_whitebg= cv2.ellipse(white,
                        center,
                        (int(x_radius(top_left_2,bottom_right_2)), int(y_radius(top_left_2,bottom_right_2)))
                        , 0, 0, 360, 0, -1)

    outer_mask = cv2.bitwise_and(white_outer_blackbg, black_ellipse_whitebg)

    #Make a mask of the ellipse that sets everything outside the ellipse to 0
    inner_mask = np.full(img.shape, 0, dtype=np.uint8)
    inner_mask = cv2.ellipse(inner_mask,
                        center,
                        (int(x_radius(top_left_2,bottom_right_2)), int(y_radius(top_left_2,bottom_right_2)))
                        , 0, 0, 360, 255, -1)

    # Normalize on blue
    # plt.imshow(
    #     cv2.bitwise_and(outer_mask, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
    # )
    # plt.show()

    # plt.imshow(
    #     cv2.bitwise_and(inner_mask, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
    # )
    # plt.show()

    # plt.imshow(
    #     cv2.bitwise_and(white_outer_blackbg, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
    # )
    # plt.show()

    if outer_bool:
        return (black , cv2.bitwise_and(white_outer_blackbg, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
                      , cv2.bitwise_and(white_outer_blackbg, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)))
    return (cv2.bitwise_and(inner_mask, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
           ,cv2.bitwise_and(outer_mask, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
           ,cv2.bitwise_and(white_outer_blackbg, cv2.normalize(img_red/img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)))

In [5]:
_, blue = cv2.imreadmulti('data/Blue-34a-dbl_0002.tiff', [], cv2.IMREAD_UNCHANGED)
_, red = cv2.imreadmulti('data/Red-34a-dbl_0002.tiff', [], cv2.IMREAD_UNCHANGED)
_, composite = cv2.imreadmulti('data/Composite-34a-dbl_0002.tiff', [], cv2.IMREAD_GRAYSCALE)

In [6]:
blue_composite = [(blue[i], composite[i]) for i in range(len(composite))]
red_composite = [(red[i], composite[i]) for i in range(len(composite))]
red_blue_composite = [(red[i], blue[i], composite[i]) for i in range(len(composite))]

In [7]:
(red1, blue1, composite1) = red_blue_composite[10]
get_pictures_norm(red1, blue1, composite1)

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8))

In [9]:
#Blue Red Composite
images = []
for idx, (rd, bl, cmp) in enumerate(red_blue_composite):
    if idx < len(red_blue_composite)*0.05 or idx > len(red_blue_composite)*0.95:
        point = get_pictures_norm(rd, bl, cmp, outer_bool=True)
    else:
        point = get_pictures_norm(rd, bl, cmp)
    images.append(point)

In [11]:
#Blue
points_blue = []
for idx, (bl, cmp) in enumerate(blue_composite[50:52]):
    if idx < len(blue_composite)*0.05 or idx > len(blue_composite)*0.95:
        point = get_intensity_line(bl, cmp, outer_bool=True)
    else:
        point = get_intensity_line(bl, cmp)
    points_blue.append(point)

# #Red
# points_red = []
# for idx, (rd, cmp) in enumerate(red_composite):
#     if idx < len(red_composite)*0.05 or idx > len(red_composite)*0.95:
#         point = get_intensity_points(rd, cmp, outer_bool=True)
#     else:
#         point = get_intensity_points(rd, cmp)
#     points_red.append(point)

NameError: name 'get_intensity_line' is not defined

In [10]:
#Blue vs red dist with images
import imageio
import os

filenames = []
inner_mean = []
outer_mean = []
for i in range(len(points_blue)):
    if i >= len(blue_composite)*0.05 and i <= len(blue_composite)*0.95:
        
        # plt.hist(points_red[i][0]/points_blue[i][0], bins=100, color='b', alpha=0.5, label='Outer',density=True)
        # plt.hist(points_red[i][1]/points_blue[i][1], bins=100, color='r', alpha=0.5, label='Inner',density=True)

        plt.hist(np.log(points_red[i][0]/points_blue[i][0]), bins=100, color='b', alpha=0.5, label='Outer',density=True)
        plt.axvline((np.log(points_red[i][0]/points_blue[i][0])).mean(), color='b', linestyle='dashed', linewidth=1)
        outer_mean.append((np.log(points_red[i][0]/points_blue[i][0])).mean())
        plt.hist(np.log(points_red[i][1]/points_blue[i][1]), bins=100, color='r', alpha=0.5, label='Inner',density=True)
        plt.axvline((np.log(points_red[i][1]/points_blue[i][1])).mean(), color='r', linestyle='dashed', linewidth=1)
        inner_mean.append((np.log(points_red[i][1]/points_blue[i][1])).mean())
        plt.legend(loc='upper right')
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
        plt.title(f'Frequency of Pixel Intensities at {i}')


    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()

def pil_grid(images, max_horiz=np.iinfo(int).max):
    n_images = len(images)
    n_horiz = min(n_images, max_horiz)
    h_sizes, v_sizes = [0] * n_horiz, [0] * (n_images // n_horiz)
    for i, im in enumerate(images):
        h, v = i % n_horiz, i // n_horiz
        h_sizes[h] = max(h_sizes[h], im.size[0])
        v_sizes[v] = max(v_sizes[v], im.size[1])
    h_sizes, v_sizes = np.cumsum([0] + h_sizes), np.cumsum([0] + v_sizes)
    im_grid = Image.new('RGB', (h_sizes[-1], v_sizes[-1]), color='white')
    for i, im in enumerate(images):
        im_grid.paste(im, (h_sizes[i % n_horiz], v_sizes[i // n_horiz]))
    return im_grid

# build gif
with imageio.get_writer('bluevsreddistwimg.gif', mode='I') as writer:
    for i in range(len(filenames)):
        if i >= len(blue_composite)*0.05 and i <= len(blue_composite)*0.95:
            imgs = [Image.fromarray(x) for x in images[i]] + [Image.open(filenames[i])]
            pil_grid(imgs).save('temp.png')
            image = imageio.imread('temp.png')
            os.remove('temp.png')
            writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)
#plt inner mean and outer mean
plt.plot(inner_mean, label='Inner', color='r')
plt.plot(outer_mean, label='Outer', color='b')

NameError: name 'points_blue' is not defined

In [ ]:
#Density scatter plot
import imageio
import os

filenames = []
for i in range(len(points_blue)):
    if i >= len(blue_composite)*0.05 and i <= len(blue_composite)*0.95:
        plt.scatter(points_blue[i][0], points_red[i][0] , c='b', marker='o', label='Outer',alpha=0.01)
        plt.scatter(points_blue[i][1], points_red[i][1], c='r', marker='o', label='Inner',alpha=0.01)
        plt.legend(loc='upper left')
        plt.xlabel('Blue')
        plt.ylabel('Red')
        ax = plt.gca()
        #set log scale
        ax.set_xscale('log')
        ax.set_yscale('log')
        plt.title(f'Blue vs Red Intensity at {i}')


    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()

def pil_grid(images, max_horiz=np.iinfo(int).max):
    n_images = len(images)
    n_horiz = min(n_images, max_horiz)
    h_sizes, v_sizes = [0] * n_horiz, [0] * (n_images // n_horiz)
    for i, im in enumerate(images):
        h, v = i % n_horiz, i // n_horiz
        h_sizes[h] = max(h_sizes[h], im.size[0])
        v_sizes[v] = max(v_sizes[v], im.size[1])
    h_sizes, v_sizes = np.cumsum([0] + h_sizes), np.cumsum([0] + v_sizes)
    im_grid = Image.new('RGB', (h_sizes[-1], v_sizes[-1]), color='white')
    for i, im in enumerate(images):
        im_grid.paste(im, (h_sizes[i % n_horiz], v_sizes[i // n_horiz]))
    return im_grid

# build gif
with imageio.get_writer('scatterwimg.gif', mode='I') as writer:
    for i in range(len(filenames)):
        if i >= len(blue_composite)*0.05 and i <= len(blue_composite)*0.95:
            imgs = [Image.fromarray(x) for x in images[i]] + [Image.open(filenames[i])]
            pil_grid(imgs).save('temp.png')
            image = imageio.imread('temp.png')
            os.remove('temp.png')
            writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

In [ ]:
#Blue
average_blue = []
for idx, (bl, cmp) in enumerate(blue_composite):
    if idx < len(blue_composite)*0.05 or idx > len(blue_composite)*0.95:
        point = get_intensity(bl, cmp, outer_bool=True)
    else:
        point = get_intensity(bl, cmp)
    average_blue.append(point)

#Red
average_red = []
for idx, (rd, cmp) in enumerate(red_composite):
    if idx < len(red_composite)*0.05 or idx > len(red_composite)*0.95:
        point = get_intensity(rd, cmp, outer_bool=True)
    else:
        point = get_intensity(rd, cmp)
    average_red.append(point)

In [ ]:
#Blue
points_blue = []
for idx, (bl, cmp) in enumerate(blue_composite):
    if idx < len(blue_composite)*0.05 or idx > len(blue_composite)*0.95:
        point = get_intensity_points(bl, cmp, outer_bool=True)
    else:
        point = get_intensity_points(bl, cmp)
    points_blue.append(point)

#Red
points_red = []
for idx, (rd, cmp) in enumerate(red_composite):
    if idx < len(red_composite)*0.05 or idx > len(red_composite)*0.95:
        point = get_intensity_points(rd, cmp, outer_bool=True)
    else:
        point = get_intensity_points(rd, cmp)
    points_red.append(point)

In [ ]:
# bluevsred.gif
import imageio
import os
from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn

def density_scatter( x , y, ax = None, sort = True, bins = 20, **kwargs )   :
    """
    Scatter plot colored by 2d histogram
    """
    if ax is None :
        fig , ax = plt.subplots()
    data , x_e, y_e = np.histogram2d( x, y, bins = bins, density = True )
    z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

    #To be sure to plot all data
    z[np.where(np.isnan(z))] = 0.0

    # Sort the points by density, so that the densest points are plotted last
    if sort :
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]

    ax.scatter( x, y, c=z, marker='o', label='Outer',alpha=0.01 )

    norm = Normalize(vmin = np.min(z), vmax = np.max(z))
    cbar = fig.colorbar(cm.ScalarMappable(norm = norm), ax=ax)
    cbar.ax.set_ylabel('Density')

    return ax


filenames = []
for i in range(len(points_blue)):
    fig , ax = plt.subplots()
    plt.scatter(points_blue[i][0],points_red[i][0] , c='b', marker='o', label='Outer',alpha=0.01)
    plt.scatter(points_blue[i][1], points_red[i][1], c='r', marker='o', label='Inner',alpha=0.01)
    plt.legend(loc='upper left')
    plt.xlabel('Blue')
    plt.ylabel('Red')
    ax = plt.gca()
    #set log scale
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.title(f'Blue vs Red Intensity at {i}')


    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()

# build gif
with imageio.get_writer('bluevsred.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

In [ ]:
points_blue, points_red

#Blue vs Red scatter plot for each pixel for each frame for both inner and outer
for i in range(len(points_blue)):
    plt.scatter(points_blue[i][0],points_red[i][0] , c='b', marker='o', label='Outer',alpha=0.01)
    plt.scatter(points_blue[i][1], points_red[i][1], c='r', marker='o', label='Inner',alpha=0.01)
    plt.legend(loc='upper left')
    plt.xlabel('Blue')
    plt.ylabel('Red')
    plt.title('Blue vs Red')
    plt.show()

In [ ]:
for i in range(len(points_blue)):
    plt.scatter(list(zip(*points_blue))[0][i], list(zip(*points_red))[0][i], c='r', marker='o', label='Inner',alpha=0.01)

ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
for i in range(len(points_blue)):
    plt.scatter(list(zip(*points_blue))[1][i], list(zip(*points_red))[1][i], c='b', marker='o', label='Outer',alpha=0.01)

ax = plt.gca()
#set log scale
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
for i in range(len(points_blue)):
    plt.scatter(list(zip(*points_blue))[0][i], list(zip(*points_red))[0][i], c='b', marker='o', label='Inner',alpha=0.01)
    plt.scatter(list(zip(*points_blue))[1][i], list(zip(*points_red))[1][i], c='r', marker='o', label='Outer',alpha=0.01)
    
ax = plt.gca()
#set log scale
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
#outer, inner
#Make a histogram of the pixel intensities for each frame
for i in range(len(points_blue)):
    if i >= len(blue_composite)*0.05 and i <= len(blue_composite)*0.95:
        plt.hist(points_red[i][0]/points_blue[i][0], bins=100, color='b', alpha=0.5, label='Outer',density=True)
        plt.hist(points_red[i][1]/points_blue[i][1], bins=100, color='r', alpha=0.5, label='Inner',density=True)
        plt.legend(loc='upper right')
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
        plt.title('Frequency of Pixel Intensities')
        plt.show()

In [ ]:
#Plot outer and inner intensities over time with labels
plt.plot([i[0] for i in average_blue], label='Outer')
plt.plot([i[1] for i in average_blue], label='Inner')
plt.legend()
plt.title('Blue')
plt.show()

In [ ]:
#Plot outer and inner intensities over time with labels
plt.plot([i[0] for i in average_red], label='Outer')
plt.plot([i[1] for i in average_red], label='Inner')
plt.legend()
plt.title('Red')
plt.show()

In [ ]:
blue_composite = [(blue[i], composite[i]) for i in range(len(composite))]
red_composite = [(red[i], composite[i]) for i in range(len(composite))]

#Blue
images = []
for idx, (bl, cmp) in enumerate(blue_composite):
    if idx < len(blue_composite)*0.05 or idx > len(blue_composite)*0.95:
        point = get_intensity_pictures(bl, cmp, outer_bool=True)
    else:
        point = get_intensity_pictures(bl, cmp)
    images.append(point)

In [ ]:
import tifffile

FRAMES = [inner for inner, outer, comp in images] + [outer for inner, outer, comp in images] +  [comp for inner, outer, comp in images]
OUT_NAME = "blue_2.tiff" # Name to save to
with tifffile.TiffWriter(OUT_NAME) as tiff:
    for img in FRAMES:
        tiff.save(img)

In [ ]:
inner_points_blue = [j for i in points_blue for j in i[0]]
inner_points_red = [j for i in points_red for j in i[0]]
outer_points_blue = [j for i in points_blue for j in i[1]]
outer_points_red = [j for i in points_red for j in i[1]]

In [ ]:
#Intrinsic and extrinsic noise for inner
#intrinsic = (1/n)*sum([0.5*(blue - red)**2 for blue, red in zip(blue_intensities, red_intensities)])/(mean(blue_intensities) * mean(red_intensities))
#extrinsic = (1/n)*sum([blue*red - mean(blue_intesity)*mean(red_intensity) for blue, red in zip(blue_intensities, red_intensities)])/(mean(blue_intensities) * mean(red_intensities))
#total = (1/n)*sum([0.5*(blue**2 - red**2) - mean(blue_intesity)*mean(red_intensity) for blue, red in zip(blue_intensities, red_intensities)])/(mean(blue_intensities) * mean(red_intensities))
inner_points_blue_mean = np.mean(inner_points_blue)
inner_points_red_mean = np.mean(inner_points_red)
inner_intrinsic = (1/len(inner_points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(inner_points_blue, inner_points_red)])/(inner_points_blue_mean * inner_points_red_mean)

outer_points_blue_mean = np.mean(outer_points_blue)
outer_points_red_mean = np.mean(outer_points_red)
outer_intrinsic = (1/len(outer_points_blue))*sum([0.5*(blue - red)**2 for blue, red in zip(outer_points_blue, outer_points_red)])/(outer_points_blue_mean * outer_points_red_mean)

In [ ]:
inner_extrinsic = (1/len(inner_points_blue))*sum([blue*red - inner_points_red_mean*inner_points_blue_mean for blue, red in zip(inner_points_blue, inner_points_red)])/(inner_points_blue_mean * inner_points_red_mean)
outer_extrinsic = (1/len(outer_points_blue))*sum([blue*red - outer_points_red_mean*outer_points_blue_mean for blue, red in zip(outer_points_blue, outer_points_red)])/(outer_points_blue_mean * outer_points_red_mean)

In [ ]:
inner_total = (1/len(inner_points_blue))*sum([0.5*(blue**2 - red**2) -  inner_points_red_mean*inner_points_blue_mean for blue, red in zip(inner_points_blue, inner_points_red)])/(inner_points_red_mean*inner_points_blue_mean )
outer_total = (1/len(outer_points_blue))*sum([0.5*(blue**2 - red**2) -  outer_points_red_mean*outer_points_blue_mean for blue, red in zip(outer_points_blue, outer_points_red)])/(outer_points_red_mean*outer_points_blue_mean )

In [ ]:
inner_intrinsic,inner_extrinsic,inner_total

In [ ]:
outer_intrinsic,outer_extrinsic,outer_total